In [8]:
import nibabel as nib
import numpy as np
from joblib import Parallel, delayed
import multiprocessing

import os
import subprocess



import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

import scipy.io as scio

import numpy as np
import os
from glob import glob
import nibabel as nb
import nibabel
#from pylab import *
import scipy.signal
from nilearn.masking import apply_mask
from nilearn.masking import unmask
import nilearn.signal
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"   # to hide os warning about h5py
import numpy as np
from glob import glob
import nibabel as nib
import re
from sklearn.decomposition import PCA
import itertools
import nilearn.masking
from nilearn import image
import scipy.ndimage
from nilearn.masking import apply_mask, unmask
import nipype.interfaces.fsl as fsl
import pickle
from joblib import Parallel, delayed
import nipype.pipeline.engine as pe          # pypeline engine 
import nipype.interfaces.spm as spm          # spm
#import nipype.interfaces.matlab as mlab
#mlab.MatlabCommand.set_default_paths('/home/miv/bhanot/soft/Ants/bin') # Set paths for Ants
#mlab.MatlabCommand.set_default_paths('/home/miv/bhanot/spm12')  # Set paths for spm
from nilearn.masking import compute_multi_epi_mask, compute_epi_mask, new_img_like
import scipy
import nipype.interfaces.ants as ants
#import skimage.exposure as ex
from nipype.interfaces.nipy import ComputeMask

import sys
sys.path.append('libs/')
#from time_series import *

from scipy.signal import butter, lfilter


def create_path(path_to_create):
    if not os.path.exists(path_to_create):
            os.makedirs(path_to_create)
            
def least_sqr(Y, A, mu_tikhonov):
    At = np.transpose(A)
    AA =  np.matmul( A, At) + mu_tikhonov * np.eye( A.shape[0] )
    tmp1 = np.linalg.inv( AA )
    tmp = np.matmul(Y,At)
    U_new = np.matmul(tmp,tmp1)
    return U_new



def BuildImagePath(pathDatasink,shorterPath,intFolder,printResults):
    print('\nCreating array of image paths...')    
    os.chdir(pathDatasink)

    arrayPath = !{'ls'}
    print('Array size: %d items.' % len(arrayPath))
    #print(array)

    arrayPathImage = arrayPath
    for index in range (len(arrayPath)):

        #print(index)
        #print(array[0])

        cmd = 'ls ' + arrayPath[index] + intFolder
        image = !{cmd}
        #print(image)


        arrayPathImage[index] = pathDatasink + '/' + arrayPathImage[index] + intFolder + '/' + image[0]
        
        if shorterPath:
            arrayPathImage[index] = pathDatasink + '/' + image[0]
        
        if printResults:
            print(arrayPathImage[index])

    if printResults:
        print(arrayPathImage)
    
    print('Array with paths made!!!\n\n')    
    
    return arrayPathImage


def PrintArray(arrayPrint):
    print('\nPrinting array...')
    print('Array length: %d' % (len(arrayPrint)))
    for index in range (len(arrayPrint)):
        print(arrayPrint[index])
    print('Array printing done!!!\n')
    
    
def SortArrayByRefArray(arrayUnsorted, arrayRef, printResults):
    arraySorted = [None]*len(arrayRef)

    for index in range (len(arrayRef)):
        indexMatch = [i for i, s in enumerate(arrayUnsorted) if arrayRef[index] in s]
        #print(indexMatch)

        if indexMatch==[]:
            print('\nAttention: Subject %s was not found!!!\n' % arrayRef[index])
            print('\nSetting path for the subject %s as None!!!\n' % arrayRef[index])                    
        
        elif (len(indexMatch) > 1):
            print('\nProblem of matching!!! Multiple indexes found!!!\n')
            break

        else:
            if printResults:
                print(index)
                print(indexMatch)
            arraySorted[index] = arrayUnsorted[indexMatch[0]]

    if printResults:    
        print(arraySorted)
    
    return arraySorted


def TapTerminalCommand(commonCmd, arraySubj, arrayPathInput, arrayPathRef, arrayPathTrans, sufix, transFormat, inverseTrans, posEraseEnd, printCmd, executeCmd):
    arrayPathOutput = [None]*len(arraySubj)
    cmdArray = [None]*len(arraySubj)
    
    
    for indexSubject in range (len(arraySubj)):
        pathOutputImage = arrayPathInput[indexSubject][:-posEraseEnd] + sufix + '_'+ arraySubj[indexSubject] + '.nii'
        if transFormat == '.nii':
            cmd = commonCmd + ' --input ' + arrayPathInput[indexSubject] + ' --output ' + pathOutputImage + ' --reference-image ' + arrayPathRef[indexSubject] + ' --transform ' + arrayPathTrans[indexSubject]
        
        elif transFormat == '.mat':
            cmd = commonCmd + ' --input ' + arrayPathInput[indexSubject] + ' --output ' + pathOutputImage + ' --reference-image ' + arrayPathRef[indexSubject] + ' --transform ' + '[' +  arrayPathTrans[indexSubject] + ',' + str(inverseTrans) + ']'
        
        arrayPathOutput[indexSubject] = pathOutputImage
        cmdArray[indexSubject] = cmd
        
        if printCmd:
            print(cmd)
            
        if executeCmd:
            !{cmd}
    return arrayPathOutput, cmdArray


def CreateConnectivityMatrix(arrayPathfMRIData, arrayPathRegisteredBrainMask, arrayPathRegisteredAtlas, pathLabels, arraySubj, pathOutput, lowCut, highCut, fs, normWeight, nLabels,  eraseFirstLabel, extImage, extMatrix, printResults, figSize, saveFiles):
    
    
    print('\n\nCreating connectivity matrix ...\n')

    for index in range (len(arrayPathfMRIData)):
        
        
        fmridata = nib.load(arrayPathfMRIData[index])
        brain_mask = nib.load(arrayPathRegisteredBrainMask[index])
        anno_masks = nib.load(arrayPathRegisteredAtlas[index])
        labelscsv = pd.read_csv(pathLabels, sep='\t', header=None) 
        
        pathOutputSubject = pathOutput + '/connectivityMatrix_' + arraySubj[index] + '_' + nLabels + 'Labels'
        print(pathOutputSubject)
        #figSize = 50
        
        if printResults:
            print("\nOriginal labels:")
            print(labelscsv)
            
        if eraseFirstLabel:
            labelscsv = labelscsv[1:]
            
            if printResults:
                print("\nRegions without first one (root):")
                print(labelscsv)
                

        print('\nMasking atlas...')
        anno4d = apply_mask(anno_masks, brain_mask)                
        print('Done!!!')
        
        if (not(nLabels=='all')):            
            nRegions = anno4d.shape
            print(nRegions)
            print(nRegions[0])
            print(int(int(nLabels)))
            print(int(nRegions[0]/2))
            print(int(nRegions[0]/2+int(nLabels)))
            
            print(anno4d.shape)
            print(anno4d[0:int(nLabels),:].shape)
            print(anno4d[int(nRegions[0]/2):int(nRegions[0]/2+int(nLabels)),:].shape)
            anno4d = np.concatenate((anno4d[0:int(nLabels),:],anno4d[int(nRegions[0]/2):int(nRegions[0]/2+int(nLabels)),:])) 
            print(anno4d.shape)
            
            labelscsv = pd.concat((labelscsv[0:int(nLabels)],labelscsv[int(nRegions[0]/2):int(nRegions[0]/2+int(nLabels))])) 
            
            #figSize = 15
                                    
            if printResults:
                print("\nReduced labels:")
                print(labelscsv)                
                                    
        
        print('\nMasking fMRI data...')
        anno4d_bin = np.where(anno4d!=0,1,0)
        fmridata_matrix = apply_mask(fmridata, brain_mask)
        print(anno4d_bin.shape)
        print('Done!!!')


        A_orig = np.copy(anno4d_bin)
        Yinit = np.copy(fmridata_matrix)
        print(Yinit.shape)
        
        print('\n Analyse of regions per voxel:')
        print('Sum of Anit') 
        print(np.sum(A_orig,0))
        print('Max of Anit') 
        print(np.max((np.sum(A_orig,0))))
        
        if normWeight:
            print('\nApplying normalization...')
            Ainit = A_orig/(np.sum(A_orig, 0)+ 1e-30)
            print('Shape Anit')
            print(Ainit.shape)
            pathOutputSubject = pathOutputSubject + '_weightNorm'
            print('Done!!!')
            
        else:
            Ainit = A_orig
            print('Shape Anit')            
            print(Ainit.shape)
            pathOutputSubject = pathOutputSubject + '_noWeightNorm'
                    
        
        print('\nApplying Butterworth filter ..')        
        Y = Yinit[1:,:]
        Y_orig_augmented = np.zeros((Y.shape[0]+200, Y.shape[1]))
        Y_orig_augmented[:200,:] = np.flipud(Y[:200,:])
        #Y_orig_augmented[200:,:] = Y_orig_augmented
        Y_orig_augmented[200:,:] = Y
        Y_orig_augmented_bp = apply_bandpass(Y_orig_augmented, lowCut, highCut, fs, order = 5)
        Y_orig_bp = Y_orig_augmented_bp[200:,:]
        
        Yinit = Y_orig_bp
        #Yinit = apply_bandpass(Yinit, lowCut, highCut, fs, order = 5)
        print('Done!!!')

        print('\nCalculating matrix...')
        U = least_sqr(Yinit, Ainit, 1e-4)

        cormat = np.corrcoef(U.T)
        # labelscsv = pd.read_csv('/media/argheesh/DATAPART1/TempArgheesh/2021.12.31_DataFromHome/2021.12.29_Test_rs-fMRI_Mouse/Atlas-template/annoVolume+2000_rsfMRI.nii.csv', sep='\t', header=None)
        col_one_list = labelscsv[0].tolist()
        col_two_list = labelscsv[1].tolist()
        print('Done!!!')

        if printResults:
            imageSize = len(labelscsv)
            %matplotlib inline
            plt.figure(figsize = (figSize,figSize))
            plt.imshow(cormat, vmin = -1, vmax = 1, cmap = 'bwr')
            plt.xticks(np.arange(imageSize),col_two_list, rotation = 90)
            plt.yticks(np.arange(imageSize),col_two_list)
            plt.colorbar()            
            
            if saveFiles:
                print('Saving image...')
                plt.savefig(pathOutputSubject + extImage)
                print('Done!!!')
            plt.show()
            

        correlation_dic = {"cormat": cormat}

        if saveFiles:
            print('Saving matrix...')
            scio.savemat(pathOutputSubject + extMatrix,correlation_dic)
            print('Done!!!')
            
        print('Connectivity matrix of subject %s made!!!' % arraySubj[index])



def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def apply_bandpass(detrended_fmri, lowcut, highcut, fs, order = 5):
    bandpassed_fmri = np.zeros_like(detrended_fmri)
    for i in range(detrended_fmri.shape[1]):
        bandpassed_fmri[:,i] = butter_bandpass_filter(detrended_fmri[:,i], lowcut, highcut, fs, order) ## CHECK ORDER
    return bandpassed_fmri


def ReplaceAffineByReference(pathImageToBeChanged,pathImageRef,arraySubj,pathSave,nameSave):
    save_name = pathImageToBeChanged
    for index in range (len(pathImageToBeChanged)):                
        print("Analyzing image: %s" % pathImageToBeChanged[index])
        print("reference image: %s" % pathImageRef[index])
        img = nib.load(pathImageToBeChanged[index])
        #affine = img.affine
        data = img.get_data()
        
        imgRef = nib.load(pathImageRef[index])
        affineRef = imgRef.affine                                
        
        test_img = nib.Nifti1Image(data, affineRef)
        test_img.set_data_dtype(np.int16)
        
        save_name[index] = pathSave + '/' + nameSave + '_' + arraySubj[index] + '.nii'
        nib.save(test_img, save_name[index])
        print("Done!!!")
        print("New image saved as: %s" % save_name[index])
    return save_name

In [4]:

# ##################################################################################################################
# Main code
# ##################################################################################################################

# ******************************************************************************************************************
# Variables 
# ******************************************************************************************************************

pathMainInputDir = '/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing'
pathAtlas4D = '/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/masks/p56_wholeBrainMask.nii'


pathOutput = '/data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/registeredBrainMasks/20230310_secondAcquisition_groupedProcessing'

arraySubjects = ['F572','F573','F574','F582','F583','F584','F585','F596','F597','F601','F602','F603','F604','M570','M571','M575','M577','M578','M579','M580','M587','M588','M589','M590','M593','M606','M607']


cmd_generalTransAppli = 'antsApplyTransforms --default-value 0 --float 0 --dimensionality 3 -e 3 -v --interpolation Linear'
# ******************************************************************************************************************
# ******************************************************************************************************************
# ******************************************************************************************************************







In [4]:
# ******************************************************************************************************************
# Execution
# ******************************************************************************************************************

os.chdir(pathOutput)

# ==================================================================================================================
# Resample atlas to the resolution of the EPI
# ==================================================================================================================

pathAtlas4D_fileFolder, pathAtlas4D_fileName = os.path.split(pathAtlas4D)
#print(pathAtlas4D_fileFolder)
#print(pathAtlas4D_fileName)

pathOutputResampledAtlas = pathOutput + '/' + pathAtlas4D_fileName[:-4] + '_resampled.nii'

cmd_resampledAtlas = cmd_generalTransAppli + ' -t identity' + \
' --input ' + pathAtlas4D + \
' --output ' +  pathOutputResampledAtlas + \
' --reference-image ' + pathMainInputDir + '/' + 'main_wf/resample_template/resampled_template.nii.gz'
print(cmd_resampledAtlas)


!{cmd_resampledAtlas}

# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================




antsApplyTransforms --default-value 0 --float 0 --dimensionality 3 -e 3 -v --interpolation Linear -t identity --input /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/masks/p56_wholeBrainMask.nii --output /data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/registeredBrainMasks/20230310_secondAcquisition_groupedProcessing/p56_wholeBrainMask_resampled.nii --reference-image /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/main_wf/resample_template/resampled_template.nii.gz
Using double precision for computations.
Input time-series image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/masks/p56_wholeBrainMask.nii
Reference image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/main_wf/resample_template/resampled_template.nii.gz
The composite transform

In [5]:
# ==================================================================================================================
# Apply inverse warp and affine transformation template to common space
# ==================================================================================================================

pathOutputResampledAtlas_tmpl2ComInvWarp = pathOutputResampledAtlas[:-4] + '_template2CommonInverseWarp.nii'

cmd_resampledAtlas_tmpl2ComInvWarp = cmd_generalTransAppli + ' --input ' + pathOutputResampledAtlas + ' --output ' + pathOutputResampledAtlas_tmpl2ComInvWarp + ' --reference-image ' + pathMainInputDir + '/commonspace_datasink/warped_template/secondlevel_template0_output_warped_image.nii.gz' + ' --transform ' + pathMainInputDir + '/transforms_datasink/template_to_common_inverse_warp/secondlevel_template0_output_1InverseWarp.nii.gz'


pathOutputResampledAtlas_tmpl2ComInvWarpAff = pathOutputResampledAtlas_tmpl2ComInvWarp[:-4] + 'Affine.nii'
cmd_resampledAtlas_tmpl2ComInvWarpAff = cmd_generalTransAppli + ' --input ' + pathOutputResampledAtlas_tmpl2ComInvWarp + ' --output ' + pathOutputResampledAtlas_tmpl2ComInvWarpAff + ' --reference-image ' + pathMainInputDir + '/commonspace_datasink/warped_template/secondlevel_template0_output_warped_image.nii.gz' + ' --transform [' + pathMainInputDir + '/transforms_datasink/template_to_common_affine/secondlevel_template0_output_0GenericAffine.mat' + ',1]'

!{cmd_resampledAtlas_tmpl2ComInvWarp}
!{cmd_resampledAtlas_tmpl2ComInvWarpAff}


#print(cmd_resampledAtlas_tmpl2ComInvWarp)
#print(cmd_resampledAtlas_tmpl2ComInvWarpAff)
# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================


Using double precision for computations.
Input time-series image: /data_network/summer_projects/alvesrod/Share/2023_mouse-dMRI/postProcessing/registeredBrainMasks/20230310_secondAcquisition_groupedProcessing/p56_wholeBrainMask_resampled.nii
Reference image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/commonspace_datasink/warped_template/secondlevel_template0_output_warped_image.nii.gz
The composite transform comprises the following transforms (in order): 
  1. /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/transforms_datasink/template_to_common_inverse_warp/secondlevel_template0_output_1InverseWarp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: LinearInterpolateImageFunction
  Applying transform(s) to timePoint/channel/dimension5 0 (out of 1).
Output warpe

In [ ]:
# ==================================================================================================================
# Application of anat2Template transformations (inverse warp and inverse affine)
# ==================================================================================================================

# -----------------------------------------------------------------------------------------------------------------
# Anat to template - References (for inverse warp and inverse affine)
# -----------------------------------------------------------------------------------------------------------------
pathDatasinkRef = pathMainInputDir + '/anat_datasink/anat_labels'

refArrayPathImage = BuildImagePath(pathDatasinkRef,0,'',0)
#PrintArray(refArrayPathImage)

refArrayPathImageOrg = SortArrayByRefArray(refArrayPathImage, arraySubjects, 0)
#PrintArray(refArrayPathImageOrg)


# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------


# -----------------------------------------------------------------------------------------------------------------
# Anat to template - Transform (inverse warp)
# -----------------------------------------------------------------------------------------------------------------
pathDatasinkTrans = pathMainInputDir + '/transforms_datasink/anat_to_template_inverse_warp'

transArrayPathImage = BuildImagePath(pathDatasinkTrans,1,'',0)
#PrintArray(transArrayPathImage)


transArrayPathImageOrg = SortArrayByRefArray(transArrayPathImage, arraySubjects, 0)
#PrintArray(transArrayPathImageOrg)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------


# -----------------------------------------------------------------------------------------------------------------
# Anat to template - Execution (inverse warp)
# -----------------------------------------------------------------------------------------------------------------
atlasResampledPath_tmpl2CommInvWarpAffine = pathOutputResampledAtlas_tmpl2ComInvWarpAff

arrayAtlasResampledPath_tmpl2CommInvWarpAffine = [atlasResampledPath_tmpl2CommInvWarpAffine]*len(arraySubjects)
#PrintArray(atlasResampledPath_tmpl2CommInvWarpAffineArray)

arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp, arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarp = TapTerminalCommand(cmd_generalTransAppli, arraySubjects, arrayAtlasResampledPath_tmpl2CommInvWarpAffine, refArrayPathImageOrg, transArrayPathImageOrg, '_anat2TemplateInverseWarp', '.nii', 1, 4, 0, 1)


PrintArray(arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarp)
PrintArray(arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------


# -----------------------------------------------------------------------------------------------------------------
# Anat to template - Transform (inverse affine)
# -----------------------------------------------------------------------------------------------------------------
pathDatasinkTrans = pathMainInputDir + '/transforms_datasink/anat_to_template_affine'

transArrayPathImage = BuildImagePath(pathDatasinkTrans,1,'',0)
#PrintArray(transArrayPathImage)


transArrayPathImageOrg = SortArrayByRefArray(transArrayPathImage, arraySubjects, 0)
#PrintArray(transArrayPathImageOrg)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------

# -----------------------------------------------------------------------------------------------------------------
# Anat to template - Execution (inverse affine)
# -----------------------------------------------------------------------------------------------------------------


arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarpAffine, arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarpAffine = TapTerminalCommand(cmd_generalTransAppli, arraySubjects, arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp, refArrayPathImageOrg, transArrayPathImageOrg, 'Affine', '.mat', 1, 9, 0, 1)


PrintArray(arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarpAffine)
PrintArray(arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarpAffine)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------

# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================


# ==================================================================================================================
# Application of bold2anat transformations (inverse warp and inverse affine)
# ==================================================================================================================

# -----------------------------------------------------------------------------------------------------------------
# Bold to anat - References (for inverse warp and inverse affine)
# -----------------------------------------------------------------------------------------------------------------
pathDatasinkRef = pathMainInputDir + '/bold_datasink/corrected_bold_ref'

refArrayPathImage = BuildImagePath(pathDatasinkRef,0,'/_run_None',0)
#PrintArray(refArrayPathImage)

refArrayPathImageOrg = SortArrayByRefArray(refArrayPathImage, arraySubjects, 0)
#PrintArray(refArrayPathImageOrg)


# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------


# -----------------------------------------------------------------------------------------------------------------
# Bold to anat - Transform (inverse warp)
# -----------------------------------------------------------------------------------------------------------------
pathDatasinkTrans = pathMainInputDir + '/transforms_datasink/inverse_warp_bold2anat'

transArrayPathImage = BuildImagePath(pathDatasinkTrans,0,'/_run_None',0)
#PrintArray(transArrayPathImage)


transArrayPathImageOrg = SortArrayByRefArray(transArrayPathImage, arraySubjects, 0)
#PrintArray(transArrayPathImageOrg)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------


# -----------------------------------------------------------------------------------------------------------------
# Bold to anat - Execution (inverse warp)
# -----------------------------------------------------------------------------------------------------------------

arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp, arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarp_bold2AnatInverseWarp = TapTerminalCommand(cmd_generalTransAppli, arraySubjects, arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarpAffine, refArrayPathImageOrg, transArrayPathImageOrg, '_bold2AnatInverseWarp', '.nii', 1, 9, 0, 1)


PrintArray(arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarp_bold2AnatInverseWarp)
PrintArray(arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------



# -----------------------------------------------------------------------------------------------------------------
# Bold to anat - Transform (inverse affine)
# -----------------------------------------------------------------------------------------------------------------
pathDatasinkTrans = pathMainInputDir + '/transforms_datasink/affine_bold2anat'

transArrayPathImage = BuildImagePath(pathDatasinkTrans,0,'/_run_None',0)
#PrintArray(transArrayPathImage)


transArrayPathImageOrg = SortArrayByRefArray(transArrayPathImage, arraySubjects, 0)
#PrintArray(transArrayPathImageOrg)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------

# -----------------------------------------------------------------------------------------------------------------
# Bold to anat - Execution (inverse affine)
# -----------------------------------------------------------------------------------------------------------------


arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarpAffine, arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarp_bold2AnatInverseWarpAffine = TapTerminalCommand(cmd_generalTransAppli, arraySubjects, arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarp, refArrayPathImageOrg, transArrayPathImageOrg, 'Affine', '.mat', 1, 9, 0, 1)


#PrintArray(arrayCmd_resampledAtlas_tmp2ComInvWarpAff_anat2TemplInvWarp_bold2AnatInverseWarpAffine)
#PrintArray(arrayAtlasResampledPath_tmpl2CommInvWarpAffine_anat2TemplateInverseWarp_bold2AnatInverseWarpAffine)
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------

# ==================================================================================================================
# ==================================================================================================================
# ==================================================================================================================

# ******************************************************************************************************************
# ******************************************************************************************************************
# ******************************************************************************************************************




In [9]:
# Change affine of brain masks

pathMasksWrongAffine = pathMainInputDir + '/bold_datasink/bold_brain_mask'
pathMasksRefAffine = pathMainInputDir + '/bold_datasink/input_bold'
pathSaveCorrectedMasks = '/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/wholebrainMasks_inputBoldAffine/20230310_secondAcquisition_groupedProcessing'
nameFile = 'wholeBrainMask_affineInputBoldImage'

pathMasksWrongAffineImage = BuildImagePath(pathMasksWrongAffine,0,'/_run_None',0)
#PrintArray(pathMasksWrongAffineImage)

pathMasksWrongAffineImageOrg = SortArrayByRefArray(pathMasksWrongAffineImage, arraySubjects, 0)
#PrintArray(pathMasksWrongAffineImageOrg)


pathMasksRefAffineImage = BuildImagePath(pathMasksRefAffine,1,'',0)
#PrintArray(pathMasksRefAffineImage)

pathMasksRefAffineImageOrg = SortArrayByRefArray(pathMasksRefAffineImage, arraySubjects, 0)
#PrintArray(pathMasksRefAffineImageOrg)



pathMasksCorrectedAffineImageOrg = ReplaceAffineByReference(pathMasksWrongAffineImageOrg,pathMasksRefAffineImageOrg,arraySubjects,pathSaveCorrectedMasks,nameFile)


#PrintArray(pathMasksCorrectedAffineImageOrg)




Creating array of image paths...
Array size: 27 items.
Array with paths made!!!



Creating array of image paths...
Array size: 27 items.
Array with paths made!!!


Analyzing image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/bold_brain_mask/_scan_info_subject_idF572.session1_split_name_MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F572-2022-11-29081351-08-T2_TurboRARE-BrukerRARE-001546_987_mrLinTrans_fslSwapDim-RAS_T2w/_run_None/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F572-2022-11-29081351-09-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold_brain_mask.nii.gz
reference image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/input_bold/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F572-2022-11-29081351-09-GE_EPI-BrukerEPI-003000_000_mrLinTrans_

<ipython-input-8-2e1445b65d19>:330: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = img.get_data()


Done!!!
New image saved as: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/wholebrainMasks_inputBoldAffine/20230310_secondAcquisition_groupedProcessing/wholeBrainMask_affineInputBoldImage_F572.nii
Analyzing image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/bold_brain_mask/_scan_info_subject_idF573.session1_split_name_MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F573-2022-11-29102918-08-T2_TurboRARE-BrukerRARE-001546_987_mrLinTrans_fslSwapDim-RAS_T2w/_run_None/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F573-2022-11-29102918-09-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold_brain_mask.nii.gz
reference image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/input_bold/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F573-2022

Done!!!
New image saved as: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/wholebrainMasks_inputBoldAffine/20230310_secondAcquisition_groupedProcessing/wholeBrainMask_affineInputBoldImage_F597.nii
Analyzing image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/bold_brain_mask/_scan_info_subject_idF601.session1_split_name_MAP6_rs-fMRI_mouse-2022_12_05_MAP6_rs-fMRI_mouse_F601-2022-12-05132147-15-T2_TurboRARE-BrukerRARE-001546_987_mrLinTrans_fslSwapDim-RAS_T2w/_run_None/MAP6_rs-fMRI_mouse-2022_12_05_MAP6_rs-fMRI_mouse_F601-2022-12-05132147-10-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold_brain_mask.nii.gz
reference image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/input_bold/MAP6_rs-fMRI_mouse-2022_12_05_MAP6_rs-fMRI_mouse_F601-2022

Done!!!
New image saved as: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/wholebrainMasks_inputBoldAffine/20230310_secondAcquisition_groupedProcessing/wholeBrainMask_affineInputBoldImage_M577.nii
Analyzing image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/bold_brain_mask/_scan_info_subject_idM578.session1_split_name_MAP6_rs-fMRI_mouse-2022_11_30_MAP6_rs-fMRI_mouse_M578-2022-12-01105722-10-T2_TurboRARE-BrukerRARE-001546_987_mrLinTrans_fslSwapDim-RAS_T2w/_run_None/MAP6_rs-fMRI_mouse-2022_11_30_MAP6_rs-fMRI_mouse_M578-2022-12-01105722-11-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold_brain_mask.nii.gz
reference image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/input_bold/MAP6_rs-fMRI_mouse-2022_11_30_MAP6_rs-fMRI_mouse_M578-2022

Done!!!
New image saved as: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/wholebrainMasks_inputBoldAffine/20230310_secondAcquisition_groupedProcessing/wholeBrainMask_affineInputBoldImage_M593.nii
Analyzing image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/bold_brain_mask/_scan_info_subject_idM606.session1_split_name_MAP6_rs-fMRI_mouse-2022_12_06_MAP6_rs-fMRI_mouse_M606-2022-12-06132718-10-T2_TurboRARE-BrukerRARE-001546_987_mrLinTrans_fslSwapDim-RAS_T2w/_run_None/MAP6_rs-fMRI_mouse-2022_12_06_MAP6_rs-fMRI_mouse_M606-2022-12-06132718-11-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold_brain_mask.nii.gz
reference image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/input_bold/MAP6_rs-fMRI_mouse-2022_12_06_MAP6_rs-fMRI_mouse_M606-2022

In [22]:
# Change affine of registered atlases

pathMasksWrongAffine = '/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/20230310_secondAcquisition_groupedProcessing'
pathMasksRefAffine = pathMainInputDir + '/bold_datasink/input_bold'
pathSaveCorrectedMasks = '/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/20230310_secondAcquisition_groupedProcessing_registeredAtlas_inputBoldAffine'
nameFile = 'p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarpAffine_inputBoldAffine'

pathMasksWrongAffineImage = BuildImagePath(pathMasksWrongAffine,1,'',0)
#PrintArray(pathMasksWrongAffineImage)

pathMasksWrongAffineImageOrg = SortArrayByRefArray(pathMasksWrongAffineImage, arraySubjects, 0)
PrintArray(pathMasksWrongAffineImageOrg)


pathMasksRefAffineImage = BuildImagePath(pathMasksRefAffine,1,'',0)
#PrintArray(pathMasksRefAffineImage)

pathMasksRefAffineImageOrg = SortArrayByRefArray(pathMasksRefAffineImage, arraySubjects, 0)
PrintArray(pathMasksRefAffineImageOrg)



pathMasksCorrectedAffineImageOrg = ReplaceAffineByReference(pathMasksWrongAffineImageOrg,pathMasksRefAffineImageOrg,arraySubjects,pathSaveCorrectedMasks,nameFile)


#PrintArray(pathMasksCorrectedAffineImageOrg)


Creating array of image paths...
Array size: 27 items.
Array with paths made!!!



Printing array...
Array length: 27
/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/20230310_secondAcquisition_groupedProcessing/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarpAffine_F572.nii
/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/20230310_secondAcquisition_groupedProcessing/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarpAffine_F573.nii
/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/20230310_secondAcquisition_groupedProcessing/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2

Array size: 27 items.
Array with paths made!!!



Printing array...
Array length: 27
/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/input_bold/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F572-2022-11-29081351-09-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold.nii.gz
/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/input_bold/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F573-2022-11-29102918-09-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold.nii.gz
/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/input_bold/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F574-2022-11-29133126-07-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bol

<ipython-input-8-2e1445b65d19>:330: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = img.get_data()


Done!!!
New image saved as: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/20230310_secondAcquisition_groupedProcessing_registeredAtlas_inputBoldAffine/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarpAffine_inputBoldAffine_F572.nii
Analyzing image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/20230310_secondAcquisition_groupedProcessing/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarpAffine_F573.nii
reference image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/input_bold/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F573-2022-11-29102918-09-GE_EPI-BrukerEPI-003000_000_mrLinTrans_f

<ipython-input-8-2e1445b65d19>:330: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = img.get_data()


...

Done!!!
New image saved as: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/20230310_secondAcquisition_groupedProcessing_registeredAtlas_inputBoldAffine/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarpAffine_inputBoldAffine_M593.nii
Analyzing image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/20230310_secondAcquisition_groupedProcessing/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarpAffine_M606.nii
reference image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/input_bold/MAP6_rs-fMRI_mouse-2022_12_06_MAP6_rs-fMRI_mouse_M606-2022-12-06132718-11-GE_EPI-BrukerEPI-003000_000_mrLinTrans_f

<ipython-input-8-2e1445b65d19>:330: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = img.get_data()


Done!!!
New image saved as: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/20230310_secondAcquisition_groupedProcessing_registeredAtlas_inputBoldAffine/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarpAffine_inputBoldAffine_M606.nii
Analyzing image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/20230310_secondAcquisition_groupedProcessing/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarpAffine_M607.nii
reference image: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/input_bold/MAP6_rs-fMRI_mouse-2022_12_06_MAP6_rs-fMRI_mouse_M607-2022-12-06154511-15-GE_EPI-BrukerEPI-003000_000_mrLinTrans_f

<ipython-input-8-2e1445b65d19>:330: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data = img.get_data()


Done!!!
New image saved as: /data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/20230310_secondAcquisition_groupedProcessing_registeredAtlas_inputBoldAffine/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarpAffine_inputBoldAffine_M607.nii


In [8]:
print(pathCorrectedBoldImageOrg)

['/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/input_bold/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F572-2022-11-29081351-09-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold.nii.gz/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F572-2022-11-29081351-09-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold.nii.gz', '/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/bold_datasink/input_bold/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F573-2022-11-29102918-09-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold.nii.gz/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F573-2022-11-29102918-09-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold.nii.gz', '/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230

In [18]:
pathFramewiseDisplacement = pathMainInputDir + '/confounds_datasink/FD_csv'

pathFramewiseDisplacementCSV = BuildImagePath(pathFramewiseDisplacement,0,'/_run_None',0)
#PrintArray(pathFramewiseDisplacementCSV)
pathFramewiseDisplacementCSVOrg = SortArrayByRefArray(pathFramewiseDisplacementCSV, arraySubjects, 0)
PrintArray(pathFramewiseDisplacementCSVOrg)


Creating array of image paths...
Array size: 27 items.
Array with paths made!!!



Printing array...
Array length: 27
/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/confounds_datasink/FD_csv/_scan_info_subject_idF572.session1_split_name_MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F572-2022-11-29081351-08-T2_TurboRARE-BrukerRARE-001546_987_mrLinTrans_fslSwapDim-RAS_T2w/_run_None/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F572-2022-11-29081351-09-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold_combined_FD_file.csv
/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/confounds_datasink/FD_csv/_scan_info_subject_idF573.session1_split_name_MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F573-2022-11-29102918-08-T2_TurboRARE-BrukerRARE-001546_987_mrLinTrans_fslSwapDim-RAS_T2w/_run_None

In [20]:
pathTransRot =  pathMainInputDir + '/confounds_datasink/confounds_csv'

pathTransRotCSV = BuildImagePath(pathTransRot,0,'/_run_None',0)
#PrintArray(pathFramewiseDisplacementCSV)
pathTransRotCSVOrg = SortArrayByRefArray(pathTransRotCSV, arraySubjects, 0)
PrintArray(pathTransRotCSVOrg)


Creating array of image paths...
Array size: 27 items.
Array with paths made!!!



Printing array...
Array length: 27
/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/confounds_datasink/confounds_csv/_scan_info_subject_idF572.session1_split_name_MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F572-2022-11-29081351-08-T2_TurboRARE-BrukerRARE-001546_987_mrLinTrans_fslSwapDim-RAS_T2w/_run_None/MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F572-2022-11-29081351-09-GE_EPI-BrukerEPI-003000_000_mrLinTrans_fslSwapDim-RAS_bold_combined_confounds.csv
/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/rabiesPreProcessing/20230310_output_secondAcquisition_groupedProcessing/confounds_datasink/confounds_csv/_scan_info_subject_idF573.session1_split_name_MAP6_rs-fMRI_mouse-2022_11_29_MAP6_rs-fMRI_mouse_F573-2022-11-29102918-08-T2_TurboRARE-BrukerRARE-001546_987_mrLinTrans_fslSwapDim-R

In [38]:
#print(pathCorrectedBoldImageOrg)
#print(pathCorrectedBoldBrainMaskImageOrg)
print(pathRegisteredAtlasImageOrg)

#print(pathLabels)
#print(arraySubjects)

['/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarpAffine_F572.nii', '/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarpAffine_F573.nii', '/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverseWarpAffine_bold2AnatInverseWarpAffine_F574.nii', '/data_network/summer_projects/alvesrod/Share/2022_mouse-rsfMRI/outputs/atlas2EPI_inverseTransformationRabies/p56_smallRegionsGrouped_v5_leftRight_4D_resampled_template2CommonInverseWarpAffine_anat2TemplateInverse